# Практическое задание к уроку 5.
## Парсинг данных. Selenium в Python

### Вариант II
#### Написать программу, которая собирает динамически загружаемые товары и складывает данные в БД. Сайт можно выбрать и свой (я выбрал сайт сети "Пятерочка" и раздел "Спецпредложения" - 'https://5ka.ru/special_offers'. В нем присутствуют динамически загружаемые товары). 

In [1]:
! pip install selenium

In [1]:
! pip install pymongo

In [28]:
# Подключение необходимых модулей и библиотек
from pymongo import MongoClient

from selenium.webdriver.chrome.service import Service
from selenium import webdriver

from selenium.webdriver.chrome.options import Options


from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.common.by import By

from selenium.webdriver.support.ui import Select

from selenium.webdriver.support.ui import WebDriverWait

from time import sleep
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

from selenium.common.exceptions import NoSuchElementException


from bs4 import BeautifulSoup as bs
import requests
import json

import pandas as pd
from pprint import pprint
from lxml import html

from tqdm import tqdm


In [68]:

s = Service('C:/Users/user/Desktop/chromedriver.exe')
browser = webdriver.Chrome(service=s)


In [69]:
# Раздел "Спецпредложения" сайта сети "Пятерочка" 
url = 'https://5ka.ru/special_offers'

In [70]:
browser.get(url)
#Увеличиваем окно на весь экран
browser.maximize_window()

In [71]:
browser.implicitly_wait(15)

In [72]:
# Убираем кнопку местоположения
button = browser.find_element(By.XPATH, '//*[@id="__layout"]/main/div[1]/div/div[1]/div/div/div[1]/div/div/div/div/div/div[1]/div[2]/button[1]')
button.click()

In [73]:
# Убираем кнопку cookies
button = browser.find_element(By.XPATH, '//*[@id="__layout"]/main/aside/div/div/div/button/div/span')
button.click()

In [74]:
# Организуем нажатие кнопки "Показать еще"
while True:
    wait = WebDriverWait(browser, 15)
    try:
        button = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[@class="add-more-btn"]')))
       
        button.click()
    except TimeoutException:
        print('Кнопку найти не удалось')
        break


Кнопку найти не удалось


In [75]:
# Товарные карточки
goods = browser.find_elements(By.XPATH, '//*[@class="product-card item"]')

In [76]:
# Бегло проверяем работоспособность
goods


[<selenium.webdriver.remote.webelement.WebElement (session="0b3e2371feb7be684bb0ac0a5aac3b9b", element="9b657a31-6984-4e54-881c-a3b591cc72f9")>,
 <selenium.webdriver.remote.webelement.WebElement (session="0b3e2371feb7be684bb0ac0a5aac3b9b", element="157d5bab-2c42-4148-9b1a-b905431e52fe")>,
 <selenium.webdriver.remote.webelement.WebElement (session="0b3e2371feb7be684bb0ac0a5aac3b9b", element="dad5d53f-b70d-4750-b665-15a8b80279ab")>,
 <selenium.webdriver.remote.webelement.WebElement (session="0b3e2371feb7be684bb0ac0a5aac3b9b", element="e0f967a3-37b6-4858-b64f-248637393031")>,
 <selenium.webdriver.remote.webelement.WebElement (session="0b3e2371feb7be684bb0ac0a5aac3b9b", element="f721ecec-bb2c-4471-bc9f-ad2572e63998")>,
 <selenium.webdriver.remote.webelement.WebElement (session="0b3e2371feb7be684bb0ac0a5aac3b9b", element="1a9a0979-9ca9-4a9a-b797-a89ffbd7006f")>,
 <selenium.webdriver.remote.webelement.WebElement (session="0b3e2371feb7be684bb0ac0a5aac3b9b", element="7bbfc3b1-f8ab-4a6f-ab03-b7

In [97]:
# Бегло проверяем работоспособность и правильность выборки товаров
goods[2].find_element(By.XPATH, './/div[@class = "item-name"]').text

'Макароны Ладные,перья, 40...'

In [104]:
goods[0].find_element(By.XPATH, './/div[@class = "discount-hint hint"]').text

'-25 %'

In [90]:
print(goods[0].find_element(By.XPATH, './/div[@class="price-right"]/span[@class="price-discount_cents"]').text) 

99


In [91]:
print(goods[0].find_element(By.XPATH, './/span[@class="from"]').text) 

от


In [111]:
print(goods[0].find_element(By.XPATH, './/span[@class="price-regular"]').text) 

249 .99


In [112]:
print(goods[0].find_element(By.XPATH, '//*[@id="__layout"]/main/div[1]/main/div/div/div[3]/div/div[1]/div[1]/div[2]/div/span').text)  #

от 189


In [110]:
len(goods)

1224

In [115]:
# Организуем хранение полученных данных в Json-файл
goods_list = []
for i in goods:
            goods_list.append(
                {
         # Для каждого товара определяем необходимые параметры                  
                    'name': i.find_element(By.XPATH, './/div[@class = "item-name"]').text, # Название товара
                    'price': i.find_element(By.XPATH, './/span[@class="price-regular"]').text, # Цена
                    'discont': i.find_element(By.XPATH, './/div[@class = "discount-hint hint"]').text, # Скидка
                                       
                }
            )

In [116]:
# Проверяем правильность
goods_list

[{'name': 'Аджика Мистраль Амца, 200...',
  'price': '249 .99',
  'discont': '-25 %'},
 {'name': 'Зубная паста Colgate Лече...',
  'price': '139 .69',
  'discont': '-22 %'},
 {'name': 'Макароны Ладные,перья, 40...',
  'price': '39 .99',
  'discont': '-13 %'},
 {'name': 'Макароны Ладные, рожки, 4...',
  'price': '39 .99',
  'discont': '-13 %'},
 {'name': 'Макароны Ладные,вермишель...',
  'price': '39 .99',
  'discont': '-13 %'},
 {'name': 'Макароны Ладные,спираль,...',
  'price': '39 .99',
  'discont': '-13 %'},
 {'name': 'Макароны Ладные, вермишел...',
  'price': '39 .99',
  'discont': '-13 %'},
 {'name': 'Печенье Белогорье Постное...',
  'price': '49 .99',
  'discont': '-21 %'},
 {'name': 'Пряники Наслада сырцовые...',
  'price': '104 .99',
  'discont': '-29 %'},
 {'name': 'Печенье Любятово Постное...',
  'price': '86 .99',
  'discont': '-20 %'},
 {'name': 'Печенье Полезный Завтрак...',
  'price': '78 .99',
  'discont': '-25 %'},
 {'name': 'Торт Русская Нива Наполео...',
  'price': '2

In [117]:
# Для наглядности заносим данные в Pandas
goods_result = pd.DataFrame(goods_list)
goods_result

,name,price,discont
0,"Аджика Мистраль Амца, 200...",249 .99,-25 %
1,Зубная паста Colgate Лече...,139 .69,-22 %
2,"Макароны Ладные,перья, 40...",39 .99,-13 %
3,"Макароны Ладные, рожки, 4...",39 .99,-13 %
4,"Макароны Ладные,вермишель...",39 .99,-13 %
...,...,...,...
1219,"Салатная смесь ""Тоскана""...",129 .99,-24 %
1220,Дск Манго 1 кг,279 .99,-8 %
1221,Дск Томат сливовидный 600...,179 .99,-9 %
1222,ДСК Грибы Шампиньоны 400...,119 .99,-17 %


In [122]:
# Экспортируем DataFrame в файл CSV
goods_result.to_csv (r' goods_5ka.csv', index= False )

In [ ]:
# Подготовка БД для приема данных

client = MongoClient('127.0.0.1', 27017)
db = client['goods_5ka']
actions_db = db.goods_5ka


In [118]:
# Заносим данные в базу в коллекцию actions_db
db.actions_db.insert_many(goods_list)

In [120]:
# Проверяем наличие заполнения данных
db.actions_db.find_one()

{'_id': ObjectId('64063cd4470eca482ea810b1'),
 'name': 'Аджика Мистраль Амца, 200...',
 'price': '249 .99',
 'discont': '-25 %'}